<a href="https://colab.research.google.com/github/VijayTaori/Lifo_step1/blob/main/Df_lifo_btc_step_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import calendar
df_open = pd.read_csv('/content/et_open_alldata.csv')
df_close = pd.read_csv('/content/et_close_alldata.csv')
df_close['Close Datetime'] = pd.to_datetime(df_close['Close_unix_timestamp'],dayfirst=True,unit='s')
df_open['Open Datetime'] = pd.to_datetime(df_open['Open_unix_timestamp'],dayfirst=True,unit='s')
df_close_btc = df_close[df_close['Close Action'] =='Buy Bitcoin']
df_open_btc = df_open[df_open['Open Action'] =='Buy Bitcoin']
df_close_btc_original = df_close_btc
df_open_btc_original = df_open_btc
df_close_btc.reset_index(inplace=True)
df_close_btc = df_close_btc.drop('index', axis=1)
df_close_btc.reset_index(inplace=True)
df_close_btc.rename(columns={'index':'Original_Close_index'},inplace=True)
df_open_btc.reset_index(inplace=True)
df_open_btc = df_open_btc.drop('index', axis=1)
df_open_btc.reset_index(inplace=True)
df_open_btc.rename(columns={'index':'Original_Open_index'},inplace=True)
data_btc = {}
df_lifo_btc = pd.DataFrame(data_btc, 
                           columns=['Original_Open_index','Original_Close_index',
                                    'Open Position ID', 'Close Position ID',
                                    'Open Units','Close Units','Spread_Eur',
                                    'Open Amount_Eur','Amount_close_Eur',
                                    'Open Datetime', 'Close Datetime',
                                    'Open Rate_Eur', 'Close Rate_Eur',
                                    'Open Action', 'Close Action', 
                                    'Open Amount','Close Rate', 'Open Rate',
                                    'Close Spread', 'Close Profit USD',
                                    'Close_Date','Open_Date',
                                    'Close_unix_timestamp','EurOpen_y', 
                                    'EurHigh_y','EurLow_y', 'EurClose_y',
                                    'EurWeekday_y', 'Open_unix_timestamp',
                                    'EurOpen_x', 'EurHigh_x','EurLow_x', 
                                    'EurClose_x','EurWeekday_x'
                                    ]
                           )

In [109]:
# Let's build a index selector for df_close_btc
close_unit_index = 0
value_close_units = df_close_btc.loc[close_unit_index, 'Close Units']
final_close_unit_index = df_open_btc.shape[0]
fruit_list = list(range(400))
serial_number_for_loop = 0
serial_number_if_loop = 0
serial_number_elif_loop_1 = 0
serial_number_elif_loop_2 = 0
serial_number_else_loop = 0
for x in fruit_list:#while close_unit_index < final_close_unit_index:
  serial_number_for_loop = serial_number_for_loop + 1
  print('For loop run number:',serial_number_for_loop)
  i = 0
  #if value_close_units == 0:
  #  i = 1
  #else:
  #  i = 0
  print('i =',i)
  value_close_units = df_close_btc.loc[close_unit_index, 'Close Units']
  # How to set up an upper limit on this variable?? use df.shape[0]
  # Choosing first row from df_close_btc where the units are = values_of_units
  # Assign it as new DF
  # change the reset the index and the drop the 'index' column
  df_latest_row_from_close = df_close_btc[close_unit_index:(close_unit_index+1)]
  df_latest_row_from_close.reset_index(inplace=True)
  current_close_index = df_latest_row_from_close.at[0,'index']
  df_latest_row_from_close = df_latest_row_from_close.drop('index',axis=1)
  # Get the datetime value from the df_latest_row_from_close
  close_unit_datetime = df_close_btc.loc[close_unit_index,'Close Datetime']
  print('close_unit_datetime:',close_unit_datetime)
  # find all the transaction datetimes from df_open that were opened before the close_unit_datetime
  open_datetimes_b4 = df_open_btc['Open Datetime'][df_open_btc['Open Datetime'] < close_unit_datetime]
  #Latest datetime beofore close datetime
  open_datetime_lifo = open_datetimes_b4.max()
  #if open_datetime_lifo == pd.NaT:
  #  break
  #else:
  #  continue
  print('open_datetime_lifo:',open_datetime_lifo)
  '''
  Remember that even if only one value for open_datetime_lifo is chosen
  this can still result in chosing more than one rows in that case last row 
  should be chosen
  '''
  # Corresponding row for the open_datetime_lifo 
  # let's make a new DF
  df_latest_last_row_open = df_open_btc[df_open_btc['Open Datetime'] == open_datetime_lifo]
  df_latest_last_row_open = df_latest_last_row_open[::-1]
  df_latest_last_row_open = df_latest_last_row_open[0:1]
  df_latest_last_row_open.reset_index(inplace=True)
  current_open_index = df_latest_last_row_open.at[0,'index']
  df_latest_last_row_open = df_latest_last_row_open.drop('index', axis=1)
  value_open_units = df_latest_last_row_open.loc[0,'Open Units']
  print('value_close_units:',value_close_units, 'value_open_units:',value_open_units)
  abs_diff_open_close_units = abs(value_close_units - value_open_units)
  print('abs_diff_open_close_units:',abs_diff_open_close_units)
  value_open_amount = df_latest_last_row_open.loc[0,'Open Amount_Eur']
  value_close_amount = df_latest_row_from_close.loc[0,'Amount_close_Eur']
  value_close_spread_eur = df_latest_row_from_close.loc[0,'Spread_Eur']
  close_Rate_eur = df_latest_row_from_close.loc[0,'Close Rate_Eur']
  open_Rate_eur = df_latest_last_row_open.loc[0,'Open Rate_Eur']
  #Remember that 'Open_index' and 'Close_index' columns host original indices
  # This is important as df_open_btc starts to drop rows the original indices will come in handy 
  # And in case of df_close_btc this should remain equal to the indices from DataFrame
  original_open_index = df_latest_last_row_open.at[0, 'Original_Open_index']
  #Remeber that earlier defined close_unit_index is the index of units from df_close_btc DataFrame
  # close_index should essentially be equal in integer value of close_unit_index
  close_index = df_latest_row_from_close.at[0, 'Original_Close_index']
  is_it_equal = close_unit_index == close_index
  print('close_unit_index for current loop is:',close_unit_index)
  print('(close_unit_index = close_index) is:',is_it_equal)
  if value_close_units > value_open_units:
    serial_number_if_loop = serial_number_if_loop + 1
    print('If_loop run number:',serial_number_if_loop)
    #First lets extract all the values from open DF df_latest_last_row_open
    newth_list_open_half = df_latest_last_row_open.to_dict('records')
    newth_dict_open_half = newth_list_open_half[0]
    #Then permanently delete the first row of the open DF df_latest_last_row_open from df_open_btc
    df_open_btc = df_open_btc.drop(current_open_index,axis=0)
    #Then reset index and drop the new column 'index' in df_open_btc
    df_open_btc.reset_index(inplace=True)
    df_open_btc = df_open_btc.drop('index',axis=1)
    #Now lets work on the Close_side
    #First define the changed values for df_latest_row_from_close
    changed_value_close_units = value_open_units
    changed_value_close_amount = changed_value_close_units * close_Rate_eur
    changed_value_close_spread = changed_value_close_units * value_close_spread_eur/value_close_units
    #First change the values of 'units' and 'amount' and 'spread' in the df_latest_row_from_close
    # These values will be reported
    df_latest_row_from_close.at[0,'Close Units'] = changed_value_close_units
    df_latest_row_from_close.at[0,'Amount_close_Eur'] = changed_value_close_amount
    df_latest_row_from_close.at[0,'Spread_Eur'] = changed_value_close_spread
    # Now lets extract all the values from df_latest_row_from_close
    newth_list_close_half = df_latest_row_from_close.to_dict('records')
    newth_dict_close_half = newth_list_close_half[0]
    newth_whole_dict = {**newth_dict_open_half,**newth_dict_close_half}
    # Now append the newth_whole_dict to the new records DataFrame df_lifo_btc
    df_lifo_btc = df_lifo_btc.append(newth_whole_dict, ignore_index=True)
    # and after that change the value of units and amount in original df_open_btc
    df_close_btc.at[close_unit_index,'Close Units'] = value_close_units - changed_value_close_units
    df_close_btc.at[close_unit_index,'Amount_close_Eur'] = value_close_amount - changed_value_close_amount
    df_close_btc.at[close_unit_index,'Spread_Eur'] = value_close_spread_eur - changed_value_close_spread
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('If_loop run number:',serial_number_if_loop,'finished')
  elif value_close_units < value_open_units:
    serial_number_elif_loop_1 = serial_number_elif_loop_1 + 1
    print('Elif_loop_1 run number:', serial_number_elif_loop_1)
    ## First lets extract all the values form close DF df_latest_row_from_close
    newth_list_close_half = df_latest_row_from_close.to_dict('records')
    newth_dict_close_half = newth_list_close_half[0]
    #First define the changed values for df_latest_row_from_close
    ## In elif statement changed values are same as we made only one change
    # and after that change the value of units and amount and spread in original df_close_btc
    df_close_btc.at[close_unit_index,'Close Units'] = 0
    df_close_btc.at[close_unit_index,'Amount_close_Eur'] = 0
    df_close_btc.at[close_unit_index,'Spread_Eur'] = 0
    #First change the values of 'units' and 'amount' and 'spread' in the df_latest_row_from_close
    # These values will be reported
    # Now lets extract all the values from df_latest_row_from_close
    ## Now lets work on the values from open DF df_latest_last_row_lifo
    ## Remember we want to get back the values in the df_latest_row_from_close 
    ## we will have to assign changed_values for open_units and open_amount
    changed_value_open_units = value_close_units
    changed_value_open_amount = changed_value_open_units * open_Rate_eur
    print('changed_value_open_units:',changed_value_open_units)
    df_latest_last_row_open.at[0,'Open Units'] = changed_value_open_units
    df_latest_last_row_open.at[0,'Open Amount_Eur'] = changed_value_open_amount
    ##After the values are changed extract the values which will then be reported
    newth_list_open_half = df_latest_last_row_open.to_dict('records')
    newth_dict_open_half = newth_list_open_half[0]
    ## and then this time change the original values in the df_open_units
    newth_whole_dict = {**newth_dict_open_half,**newth_dict_close_half}
    # Now append the newth_whole_dict to the new records DataFrame df_lifo_btc
    df_lifo_btc = df_lifo_btc.append(newth_whole_dict, ignore_index=True)
    # and after that change the value of units and amount in original df_open_btc
    df_open_btc.at[current_open_index,'Open Units'] = value_open_units - changed_value_open_units
    df_open_btc.at[current_open_index,'Open Amount_Eur'] = value_open_amount - changed_value_open_amount
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('Elif_loop_1 run number:', serial_number_elif_loop_1, 'finished')
  elif value_close_units == value_open_units:
    serial_number_elif_loop_2 = serial_number_elif_loop_2 + 1
    if value_close_units == 0:
      print('We are running value_close_units == 0 condition')
      print('This is in the Elif_loop_number_2:',serial_number_elif_loop_2)
      #Then permanently delete the first row of the open DF df_latest_last_row_open from df_open_btc
      df_open_btc = df_open_btc.drop(current_open_index,axis=0)
      #Then reset index and drop the new column 'index' in df_open_btc
      df_open_btc.reset_index(inplace=True)
      df_open_btc = df_open_btc.drop('index',axis=1)
      print('We are breaking')
      break
    else:
      #First lets extract all the values from open DF df_latest_last_row_open
      newth_list_open_half = df_latest_last_row_open.to_dict('records')
      newth_dict_open_half = newth_list_open_half[0]
      #Then permanently delete the first row of the open DF df_latest_last_row_open from df_open_btc
      df_open_btc = df_open_btc.drop(current_open_index,axis=0)
      #Then reset index and drop the new column 'index' in df_open_btc
      df_open_btc.reset_index(inplace=True)
      df_open_btc = df_open_btc.drop('index',axis=1)
      ### Second lets extract all the values form close DF df_latest_row_from_close
      newth_list_close_half = df_latest_row_from_close.to_dict('records')
      newth_dict_close_half = newth_list_close_half[0]
      ## and then this time change the original values in the df_open_units
      newth_whole_dict = {**newth_dict_open_half,**newth_dict_close_half}
      # Now append the newth_whole_dict to the new records DataFrame df_lifo_btc
      df_lifo_btc = df_lifo_btc.append(newth_whole_dict, ignore_index=True)
      #First define the changed values for df_latest_row_from_close
      changed_value_close_units = value_open_units
      changed_value_close_amount = changed_value_close_units * close_Rate_eur
      changed_value_close_spread = changed_value_close_units * value_close_spread_eur/value_close_units
      # and after that change the value of units and amount in original df_open_btc
      df_close_btc.at[close_unit_index,'Close Units'] = value_close_units - changed_value_close_units
      df_close_btc.at[close_unit_index,'Amount_close_Eur'] = value_close_amount - changed_value_close_amount
      df_close_btc.at[close_unit_index,'Spread_Eur'] = value_close_spread_eur - changed_value_close_spread
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('Elif_loop_2 run number:', serial_number_elif_loop_2, 'finished')
  else:
    serial_number_else_loop = serial_number_else_loop + 1
    print('Do we need to write more code? The answer is a: BIG YES!!')
    print('Or on second thought this is a silly mistake!!!')
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('serial_number_else_loop:', serial_number_else_loop, 'finished')
    print('We are breaking here!!')
    break
  value_close_units = df_close_btc.loc[close_unit_index, 'Close Units']
  if value_close_units < 1.0e-10:
    i = 1
  else:
    i = 0
  print('i =',i)
  print('close_unit_index for current loop was:',close_unit_index)
  close_unit_index = close_unit_index + i
  print('close_unit_index for next loop is:',close_unit_index)
  print('Total if loops:',serial_number_if_loop)
  print('Total elif_1 loops:',serial_number_elif_loop_1)
  print('Total elif_2 loops:',serial_number_elif_loop_2)
  print('Total else loops:',serial_number_else_loop)
  print('For loop run number:',serial_number_for_loop,'finished')
  print('****************************************')

Streaming output truncated to the last 5000 lines.
close_unit_index for current loop was: 91
close_unit_index for next loop is: 91
Total if loops: 81
Total elif_1 loops: 82
Total elif_2 loops: 8
Total else loops: 0
For loop run number: 171 finished
****************************************
For loop run number: 172
i = 0
close_unit_datetime: 2020-08-31 19:14:32
open_datetime_lifo: 2020-08-18 10:11:02
value_close_units: 0.005082999999999999 value_open_units: 0.014865
abs_diff_open_close_units: 0.009782
close_unit_index for current loop is: 91
(close_unit_index = close_index) is: True
Elif_loop_1 run number: 83
changed_value_open_units: 0.005082999999999999
value_close_units: 0.005082999999999999
value_open_units: 0.014865
close_unit_index: 91
Elif_loop_1 run number: 83 finished
i = 1
close_unit_index for current loop was: 91
close_unit_index for next loop is: 92
Total if loops: 81
Total elif_1 loops: 83
Total elif_2 loops: 8
Total else loops: 0
For loop run number: 172 finished
**********

KeyError: ignored

In [110]:
print('df_lifo_btc sum of open units:',df_lifo_btc['Open Units'].sum())
print('df_lifo_btc sum of close units:',df_lifo_btc['Close Units'].sum())
print('df_open_btc_original sum of open units:',df_open_btc_original['Open Units'].sum())
print('df_close_btc_original sum of close units:',df_close_btc_original['Close Units'].sum())

df_lifo_btc sum of open units: 4.691587
df_lifo_btc sum of close units: 4.691587
df_open_btc_original sum of open units: 4.691587
df_close_btc_original sum of close units: 4.691587


In [107]:
df_close_btc_original[0:11]['Close Units'].sum()

0.7808439999999999

In [111]:
df_open_btc

,Original_Open_index,Open Position ID,Open Action,Open Amount,Open Units,Open Datetime,Open Rate,Open_Date,Open_unix_timestamp,EurOpen_x,EurHigh_x,EurLow_x,EurClose_x,EurWeekday_x,Open Amount_Eur,Open Rate_Eur
0,0,599295474,Buy Bitcoin,100.0,4.336809e-17,2020-04-17 13:09:20,7128.14,17/04/2020 00:00,1587128960,1.086012,1.089325,1.081268,1.085847,Friday,-0.000625,6592.389676
1,75,636388600,Buy Bitcoin,25.0,1.301043e-18,2020-06-02 18:27:42,9575.15,02/06/2020 00:00,1591122462,1.112557,1.119570,1.111568,1.112595,Tuesday,-0.000645,8614.092885


In [113]:
df_close_btc

,Original_Close_index,Close Position ID,Close Action,Close Units,Close Datetime,Close Rate,Close Spread,Close Profit USD,Close_Date,Close_unix_timestamp,EurOpen_y,EurHigh_y,EurLow_y,EurClose_y,EurWeekday_y,Close Rate_Eur,Amount_close_Eur,Spread_Eur
0,0,599996081,Buy Bitcoin,0.000000e+00,2020-05-01 11:44:24,9027.00,0.96,27.72,01/05/2020 00:00,1588333464,1.094475,1.101625,1.093542,1.094547,Friday,8194.258482,0.000000e+00,0.000000e+00
1,1,610187469,Buy Bitcoin,0.000000e+00,2020-05-01 20:55:45,8685.48,3.83,-30.73,01/05/2020 00:00,1588366545,1.094475,1.101625,1.093542,1.094547,Friday,7884.243731,0.000000e+00,0.000000e+00
2,2,610190271,Buy Bitcoin,0.000000e+00,2020-05-01 20:56:05,8692.19,3.85,-28.20,01/05/2020 00:00,1588366565,1.094475,1.101625,1.093542,1.094547,Friday,7890.334733,0.000000e+00,0.000000e+00
3,3,609940706,Buy Bitcoin,0.000000e+00,2020-05-01 20:57:04,8699.43,7.79,-25.13,01/05/2020 00:00,1588366624,1.094475,1.101625,1.093542,1.094547,Friday,7896.906842,0.000000e+00,0.000000e+00
4,4,609941239,Buy Bitcoin,0.000000e+00,2020-05-01 20:57:54,8709.82,7.37,-31.33,01/05/2020 00:00,1588366674,1.094475,1.101625,1.093542,1.094547,Friday,7906.338364,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,199,1368128599,Buy Bitcoin,0.000000e+00,2022-05-12 09:16:23,27867.01,7.21,-425.77,12/05/2022 00:00,1652346983,1.051248,1.052975,1.037355,1.051248,Thursday,26465.025290,0.000000e+00,0.000000e+00
200,200,1370705010,Buy Bitcoin,0.000000e+00,2022-05-12 09:16:27,27862.74,3.61,-218.86,12/05/2022 00:00,1652346987,1.051248,1.052975,1.037355,1.051248,Thursday,26460.970110,0.000000e+00,0.000000e+00
201,201,1353186945,Buy Bitcoin,0.000000e+00,2022-05-12 09:16:29,27816.47,0.91,-55.21,12/05/2022 00:00,1652346989,1.051248,1.052975,1.037355,1.051248,Thursday,26417.027940,0.000000e+00,0.000000e+00
202,202,1361963252,Buy Bitcoin,0.000000e+00,2022-05-12 09:16:32,27799.73,0.84,-52.98,12/05/2022 00:00,1652346992,1.051248,1.052975,1.037355,1.051248,Thursday,26401.130130,0.000000e+00,0.000000e+00


In [74]:
df_lifo_btc

,Original_Open_index,Original_Close_index,Open Position ID,Close Position ID,Open Units,Close Units,Spread_Eur,Open Amount_Eur,Amount_close_Eur,Open Datetime,...,EurHigh_y,EurLow_y,EurClose_y,EurWeekday_y,Open_unix_timestamp,EurOpen_x,EurHigh_x,EurLow_x,EurClose_x,EurWeekday_x
0,32,0,611192720,599996081,0.002861,0.002861,0.177512,23.071989,23.443774,2020-04-30 19:14:44,...,1.101625,1.093542,1.094547,Friday,1588274084,1.087666,1.096996,1.083565,1.087725,Thursday
1,31,0,610410583,599996081,0.002867,0.002867,0.177884,23.071989,23.492939,2020-04-30 10:04:41,...,1.101625,1.093542,1.094547,Friday,1588241081,1.087666,1.096996,1.083565,1.087725,Thursday
2,30,0,610190271,599996081,0.008421,0.008421,0.522485,71.273913,69.003851,2020-04-30 04:14:49,...,1.101625,1.093542,1.094547,Friday,1588220089,1.087666,1.096996,1.083565,1.087725,Thursday
3,30,1,610190271,610187469,0.050460,0.050460,3.013455,427.081054,397.838939,2020-04-30 04:14:49,...,1.101625,1.093542,1.094547,Friday,1588220089,1.087666,1.096996,1.083565,1.087725,Thursday
4,29,1,610190123,610187469,0.008187,0.008187,0.488925,69.024397,64.548303,2020-04-30 04:13:53,...,1.101625,1.093542,1.094547,Friday,1588220033,1.087666,1.096996,1.083565,1.087725,Thursday
5,29,2,610190123,610190271,0.002759,0.002759,0.164969,23.263560,21.769434,2020-04-30 04:13:53,...,1.101625,1.093542,1.094547,Friday,1588220033,1.087666,1.096996,1.083565,1.087725,Thursday
6,28,2,610188766,610190271,0.010912,0.010912,0.652461,92.287957,86.099333,2020-04-30 04:08:23,...,1.101625,1.093542,1.094547,Friday,1588219703,1.087666,1.096996,1.083565,1.087725,Thursday
7,27,2,610188710,610190271,0.010904,0.010904,0.651982,92.287957,86.036210,2020-04-30 04:08:05,...,1.101625,1.093542,1.094547,Friday,1588219685,1.087666,1.096996,1.083565,1.087725,Thursday
8,26,2,610188689,610190271,0.010883,0.010883,0.650727,92.287957,85.870513,2020-04-30 04:07:57,...,1.101625,1.093542,1.094547,Friday,1588219677,1.087666,1.096996,1.083565,1.087725,Thursday
9,25,2,610187738,610190271,0.010866,0.010866,0.649710,92.287957,85.736377,2020-04-30 04:02:13,...,1.101625,1.093542,1.094547,Friday,1588219333,1.087666,1.096996,1.083565,1.087725,Thursday
